In [66]:
!pip install ipynb
!pip install nltk
#!pip install model

In [67]:
import json
#from nltk_reqs import tokenize, stem, bag_of_words
import nltk
nltk.download('punkt')
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/surbhigupta/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [104]:
#neural network model code

class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.l2 = nn.Linear(hidden_size, hidden_size)
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
       #no activation and no softmax 
        return out

# NLP Concepts (stemming,  tokenization, bag of words)

In [105]:
def tokenize(sentence):
    return nltk.word_tokenize(sentence)
def stem(word):
    return stemmer.stem(word.lower())
def bag_of_words(tokenized_sentence,  all_words):
    """
    sentence = ["hello", "how", "are", "you"]
    words = ["hi", "hello", "I", "you", "bye", "thank", "cool"]
    bag = [0,1,0,1,0,0,0]
    """
    
    tokenized_sentence = [stem(w) for w in tokenized_sentence]
    bag = np.zeros(len(all_words), dtype = np.float32)
    for idx, w in enumerate(all_words):
        if w in tokenized_sentence:
            bag[idx] = 1.0
    return bag

sentence = ["hello", "how", "are", "you"]
words = ["hi", "hello", "I", "you", "bye", "thank", "cool"]
bag = bag_of_words(sentence, words)
print(bag)

[0. 1. 0. 1. 0. 0. 0.]


# Create Training Data

In [106]:
with open('intents.json', 'r') as f:
    intents = json.load(f)

In [107]:
all_words = []
tags = []
xy = []
for intent in intents['intents']:
    tag = intent['tag']
    tags.append(tag)
    for pattern in intent['patterns']:
        w = tokenize(pattern)
        all_words.extend(w)
        xy.append((w, tag))

ignore_words = ['?', '!', '.', ',']
print(all_words)

['google', 'search', 'internet', 'Hi', 'there', 'How', 'are', 'you', 'Is', 'anyone', 'there', '?', 'Hey', 'Hola', 'Hello', 'Good', 'day', 'Namaste', 'yo', 'Bye', 'See', 'you', 'later', 'Goodbye', 'Get', 'lost', 'Till', 'next', 'time', 'bbye', 'Thanks', 'Thank', 'you', 'That', "'s", 'helpful', 'Awesome', ',', 'thanks', 'Thanks', 'for', 'helping', 'me', 'How', 'you', 'could', 'help', 'me', '?', 'What', 'you', 'can', 'do', '?', 'What', 'help', 'you', 'provide', '?', 'How', 'you', 'can', 'be', 'helpful', '?', 'What', 'support', 'is', 'offered', 'Tell', 'me', 'a', 'joke', 'Joke', 'Make', 'me', 'laugh', 'Who', 'are', 'you', 'what', 'are', 'you', 'What', 'is', 'the', 'time', 'what', 'is', 'the', 'date', 'date', 'time', 'tell', 'me', 'the', 'date', 'day', 'what', 'day', 'is', 'is', 'today', 'Whats', 'up', 'Wazzup', 'How', 'are', 'you', 'sup', 'How', 'you', 'doing', 'haha', 'lol', 'rofl', 'lmao', 'thats', 'funny', 'Who', 'made', 'you', 'who', 'designed', 'you', 'who', 'programmed', 'you', 'you'

In [108]:
all_words = [stem(w) for w in all_words if w not in ignore_words]
print(all_words)

['googl', 'search', 'internet', 'hi', 'there', 'how', 'are', 'you', 'is', 'anyon', 'there', 'hey', 'hola', 'hello', 'good', 'day', 'namast', 'yo', 'bye', 'see', 'you', 'later', 'goodby', 'get', 'lost', 'till', 'next', 'time', 'bbye', 'thank', 'thank', 'you', 'that', "'s", 'help', 'awesom', 'thank', 'thank', 'for', 'help', 'me', 'how', 'you', 'could', 'help', 'me', 'what', 'you', 'can', 'do', 'what', 'help', 'you', 'provid', 'how', 'you', 'can', 'be', 'help', 'what', 'support', 'is', 'offer', 'tell', 'me', 'a', 'joke', 'joke', 'make', 'me', 'laugh', 'who', 'are', 'you', 'what', 'are', 'you', 'what', 'is', 'the', 'time', 'what', 'is', 'the', 'date', 'date', 'time', 'tell', 'me', 'the', 'date', 'day', 'what', 'day', 'is', 'is', 'today', 'what', 'up', 'wazzup', 'how', 'are', 'you', 'sup', 'how', 'you', 'do', 'haha', 'lol', 'rofl', 'lmao', 'that', 'funni', 'who', 'made', 'you', 'who', 'design', 'you', 'who', 'program', 'you', 'you', 'are', 'dumb', 'shut', 'up', 'idiot', 'what', 'are', 'you'

In [109]:
all_words = sorted(set(all_words))
tags = sorted(set(tags))
print(tags)

['Identity', 'activity', 'age', 'appreciate', 'contact', 'covid19', 'cricket', 'datetime', 'exclaim', 'goodbye', 'google', 'greeting', 'greetreply', 'haha', 'inspire', 'insult', 'jokes', 'karan', 'news', 'nicetty', 'no', 'noanswer', 'options', 'programmer', 'riddle', 'song', 'suggest', 'thanks', 'timer', 'weather', 'whatsup']


In [110]:
X_train = []
y_train = []
for (pattern_sentence, tag) in xy:
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    
    label = tags.index(tag)
    y_train.append(label) #CrossEntropyLoss
    
X_train = np.array(X_train)
y_train = np.array(y_train)

In [115]:
class ChatDataset(Dataset):
    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train
     
    #dataset[idx]
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]
    
    def __len__(self):
        return self.n_samples
#Hyperparameters
batch_size = 8
hidden_size = 0
output_size = len(tags)
input_size = len(X_train[0])
learning_rate = 0.001
num_epochs = 1000
#print(input_size, len(all_words))
#print(output_size, tags)


dataset = ChatDataset()
train_loader = DataLoader(dataset = dataset, batch_size = batch_size, shuffle=True, num_workers=2)

In [119]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# PyTorch model and training

In [120]:
model = NeuralNet(input_size, hidden_size, output_size)

In [121]:
#loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [122]:
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        wordsd = words.to(device)
        labels = labels.to(device)
        
        #forward
        outputs = model(words)
        loss = criterion(outputs, labels)
        
        #backward and optimizer step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if (epoch +1) % 100 == 0:
        print(f'epochs {epoch+1}/{num_epochs}, loss ={loss.item():.4f}')

print(f'final loss, loss={loss.item():.4f}')

RuntimeError: DataLoader worker (pid(s) 25767, 25769) exited unexpectedly